In [17]:
!pip install pandas

!pip install BeautifulSoup4
!pip install selenium
!pip install selenium --upgrade
!pip install webdriver-manager

In [76]:
import os
from datetime import datetime
import time
import requests
import re

import pandas as pd

from bs4 import BeautifulSoup

import selenium


from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select


In [129]:
# headless 모드를 위해 아래 3줄의 주석을 풀어주세요.

# option = webdriver.ChromeOptions()
# option.add_argument("--headless")

service = Service(executable_path='chromedriver.exe')
browser = webdriver.Chrome(service=service) #, options=option)

url = r"https://www.catch.co.kr/"

browser.maximize_window()
print(browser.get_window_size())

browser.get(url)

{'width': 1552, 'height': 832}


In [130]:
from accounts import ct

ID = ct.ID
PW = ct.PW

In [131]:
# 로그인

browser.find_element(By.CSS_SELECTOR, "a.login").click()

browser.find_element(By.CSS_SELECTOR, "input#id_login").send_keys(ID)
time.sleep(1)
browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(PW)

browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(Keys.ENTER)

In [177]:
# 기업 검색 (input)
query = browser.find_element(By.CSS_SELECTOR, "input")

query.send_keys(input("리뷰 확인하기: "))
query.send_keys(Keys.ENTER)

In [133]:
### 팝업 뜨면 끄기 (예외 처리)

try : 
    browser.find_element(By.XPATH, r'//*[@id="layerInterest4"]/div/button[2]/span').click()

except : 
    pass

In [69]:
### 리뷰가 없어도 예외 처리

In [ ]:
#commentList > div.top > div > 

In [ ]:
# 검색한 기업(최상위 노출) 클릭 
browser.find_element(By.CSS_SELECTOR, "p.name > a").click()
time.sleep(2)

In [182]:
# '현직자 리뷰' 탭 클릭
browser.find_element(By.XPATH, r'//*[@id="Contents"]/div[1]/div[1]/div[2]/ul/li[4]/a').click()
time.sleep(2)

In [183]:
# 정규직, 비정규직 구분 없이 전부 조회
select = Select(browser.find_element(By.CSS_SELECTOR, 'select.select9.ml5'))
select.select_by_index(0)
time.sleep(1)

In [185]:
# 현직자, 퇴사자 구분 없이 전부 조회
browser.find_element(By.CSS_SELECTOR, 'div.top > div > span > label').click()
time.sleep(1)

In [186]:
# 긍정 리뷰만 조회
good_rv = browser.find_element(By.CSS_SELECTOR, "li:nth-child(1) > p:nth-child(4)").text
good_rv = good_rv.replace(good_rv[:5], "").replace('\n', ' ') 
good_rv

'정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준다. 짬순으로 정치순으로 진급 가능'

In [187]:
# 부정 리뷰만 조회
bad_rv = browser.find_element(By.CSS_SELECTOR, "li:nth-child(1) > p:nth-child(5)").text
bad_rv = bad_rv.replace(bad_rv[:4], "").replace('\n', ' ') 
bad_rv

'정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사원들의 업무를 독박으로 받아야 한다. 고여 있어서 최신 기술을 배울 수 없고 재택 근무가 불가능함'

In [188]:
good_df = pd.DataFrame({"긍정적 평가" : [good_rv]})
good_df

,긍정적 평가
0,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...


In [189]:
bad_df = pd.DataFrame({"부정적 평가" : [bad_rv]})
bad_df

,부정적 평가
0,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...


In [190]:
good_df.join(bad_df)

,긍정적 평가,부정적 평가
0,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...


In [167]:
# 한 페이지를 긁어 오는 함수 

def pager():

    good_lst = []
    bad_lst = []

    for i in range(1, 5) : 

        grv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({i}) > p:nth-child(4)").text
        brv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({i}) > p:nth-child(5)").text

        grv_clean = grv.replace(grv[:5], "").replace('\n', ' ') 
        brv_clean = brv.replace(brv[:4], "").replace('\n', ' ') 

        good_lst.append(grv_clean)
        bad_lst.append(brv_clean)


In [ ]:
#commentList > div.pd_type2 > p > a.ico.next

In [191]:
n_btn = browser.find_element(By.CSS_SELECTOR, 'a.ico.next')

i = 0 

while i < 151 : 
    n_btn.click()
    time.sleep(2)
    i += 1

    if i == 150 :
        break

ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:void(0);" class="ico next">...</a> is not clickable at point (732, 42). Other element would receive the click: <span class="btn_interest gtm-comp-common-like btn_interest">...</span>
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00ECA813+48355]
	(No symbol) [0x00E5C4B1]
	(No symbol) [0x00D65358]
	(No symbol) [0x00D95062]
	(No symbol) [0x00D93B98]
	(No symbol) [0x00D9239B]
	(No symbol) [0x00D9179E]
	(No symbol) [0x00D89F5C]
	(No symbol) [0x00DAA73C]
	(No symbol) [0x00D89A36]
	(No symbol) [0x00DAAA94]
	(No symbol) [0x00DBC922]
	(No symbol) [0x00DAA536]
	(No symbol) [0x00D882DC]
	(No symbol) [0x00D893DD]
	GetHandleVerifier [0x0112AABD+2539405]
	GetHandleVerifier [0x0116A78F+2800735]
	GetHandleVerifier [0x0116456C+2775612]
	GetHandleVerifier [0x00F551E0+616112]
	(No symbol) [0x00E65F8C]
	(No symbol) [0x00E62328]
	(No symbol) [0x00E6240B]
	(No symbol) [0x0

In [119]:
good_lst

['복지가 정말 좋다 밥이 정말 맛있다 밥이 최고의 복지임',
 '휴가를 눈치 보지 않고 정말 자유롭게 사용 가능, 사내 복지 시설이 너무 좋다. 구내 식당도 최고다,',
 '밥 맛있고, 출근 시간 유연하고, 복지좋고, 팀바팀이지만 팀분위기 좋았고, 워크샵 통해서 부원들과 많이 친해질 수 있음',
 '진리의 부바부 팀바팀. 잘만났다면 어지간해서 눈치안보고 편하게 다닐 수 있음']

In [113]:
bad_lst

['팀바팀이 너무 강한 회사라고 생각됨 경영진에 대한 불만이 많은 곳임',
 '어느 회사나 마찬가지겠지만 라인이 있다. 아닌듯 하지만 팀 내 실권자를 잘 따라야 하는 분위기다.',
 '체계가 없다. 매우 비효율적으로 일을 시킨다. 유의미한 작업인 지 모르겠다, 반복적으로 같은 내용을 한 느낌이다.',
 '정치질이 상당해서 잘못걸린 경우 빠른 퇴사가 정답인 경우가 많다']

In [112]:
n_btn = 'div.pd_type2 > p > a.ico.next'

i = 0

try : 
    n_btn.click()

while : 
    n_btn = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({i}) > p:nth-child(5)")

    

SyntaxError: expected 'except' or 'finally' block (2022631482.py, line 8)